Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# AOAI Finetuning and Benchmarking

The objective of this notebook is to illustrate how to use the component-based approach to evaluate a fine-tuned AOAI model on a user-provided dataset. It walks you through all stages of the process starting with model fine-tuning and concluding with metrics calculation.

## 1. Prerequisites

### 1.1. Compute with Managed Indentity

The AOAI Fine-tuning component requires specific permissions to access various resources, which must be granted prior to job submission. For authentication, permissions will be added to the User Managed Identity (UMI) and attached to the compute instance where the component will run. Detailed instructions for creating the managed identity can be found[here](https://learn.microsoft.com/en-us/entra/identity/managed-identities-azure-resources/how-manage-user-assigned-managed-identities?pivots=identity-mi-methods-azp). The instruction on how to assign managed identity to the compute can be found [here](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-identity-based-service-authentication?view=azureml-api-2&tabs=cli#user-assigned-managed-identity). See the following [link](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/role-based-access-control) for more details on the role-based access control for Azure OpenAI Service.

The Following are the minimum permissions that need to be attached to the User Managed Identity (UMI):
- `Cognitive service contributor` role over `Azure OpenAI resource`
- `Cognitive service user` role over `Azure OpenAI resource`


Furthermore, users can provide dataset URIs as inputs for training and validation data. If users intend to provide non-public data URIs, they must first store them in the workspace's associated Key Vault and then pass the Key Vault key as an input. In this case, the managed identity will need permissions to access both the workspace and the Key Vault:
- `Reader` role over AML workspace
- `Get Secret` permission over workspace' associated key vault

Key Vault access configuration can be of two types: RBAC and Vault access policy. If the Key Vault supports RBAC authorization, assign the `Key Vault Secrets User` role to the UMI over the Key Vault's scope. Otherwise, navigate to the Access Policies tab in the Key Vault resource in the Azure portal and create a new access policy to grant the "Get Secret" permission to the UMI.

In [12]:
managed_identity_resource_id = "subscriptions/72c03bf3-4e69-41af-9532-dfcdc3eefef4/resourceGroups/aml-benchmarking/providers/Microsoft.ManagedIdentity/userAssignedIdentities/finetuning-umi"

## 2. Setup

In [13]:
# Import required libraries
import os
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient, Output
from azure.ai.ml.dsl import pipeline
import pandas as pd

### 2.1. Configure workspace details and get a handle to the workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

To connect to a workspace, we need identifier parameters - a subscription, resource group and workspace name. We will use these details in the `MLClient` from `azure.ai.ml` to get a handle to the required Azure Machine Learning workspace. We use the default [default azure authentication](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity.defaultazurecredential?view=azure-python) for this tutorial. Check the [configuration notebook](https://github.com/Azure/MachineLearningNotebooks/blob/master/configuration.ipynb) for more details on how to configure credentials and connect to a workspace.

In [14]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

subscription_id = "72c03bf3-4e69-41af-9532-dfcdc3eefef4"
resource_group = "aml-benchmarking"
workspace_name = "chirag-ws"

ml_client = MLClient(credential, subscription_id, resource_group, workspace_name)

### 2.2 Show Azure ML Workspace information

In [15]:
ws = ml_client.workspaces.get(name=ml_client.workspace_name)

output = {}
output["Workspace"] = ml_client.workspace_name
output["Subscription ID"] = ml_client.connections._subscription_id
output["Resource Group"] = ws.resource_group
output["Location"] = ws.location
pd.DataFrame(data=output, index=[""]).T

,
Workspace,chirag-ws
Subscription ID,72c03bf3-4e69-41af-9532-dfcdc3eefef4
Resource Group,aml-benchmarking
Location,eastus


## 3. Compute

### Create or Attach existing AmlCompute with managed identity

You will need to create a compute target for your pipeline run. In this tutorial, you will create AmlCompute as your compute resource with managed identity.

> Note that if you have an AzureML Data Scientist role, you will not have permission to create compute resources. Talk to your workspace or IT admin to create the compute targets described in this section, if they do not already exist.

**Creation of AmlCompute takes approximately 5 minutes.**

If the AmlCompute with that name is already in your workspace, this code will skip the creation process.
As with other Azure services, there are limits on certain resources (e.g. AmlCompute) associated with the Azure Machine Learning service. Please read [this article](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-manage-quotas) on the default limits and how to request more quota.

In [16]:
from azure.ai.ml.entities import (
    ManagedIdentityConfiguration,
    IdentityConfiguration,
    AmlCompute,
)
from azure.ai.ml.constants import ManagedServiceIdentityType

# Create an identity configuration from the user-assigned managed identity
managed_identity = ManagedIdentityConfiguration(
    resource_id=managed_identity_resource_id
)
identity_config = IdentityConfiguration(
    type=ManagedServiceIdentityType.USER_ASSIGNED,
    user_assigned_identities=[managed_identity],
)

# specify aml compute name.
cpu_compute_target = "aoai-compute"

try:
    compute = ml_client.compute.get(cpu_compute_target)
except Exception:
    print("Creating a new cpu compute target...")
    # Pass the identity configuration
    compute = AmlCompute(
        name=cpu_compute_target,
        size="STANDARD_DS3_V2",
        min_instances=0,
        max_instances=4,
        identity=identity_config,
    )
    poller = ml_client.compute.begin_create_or_update(compute)
    poller.wait()
print(compute)

enable_node_public_ip: true
id: /subscriptions/72c03bf3-4e69-41af-9532-dfcdc3eefef4/resourceGroups/aml-benchmarking/providers/Microsoft.MachineLearningServices/workspaces/chirag-ws/computes/aoai-compute
identity:
  tenant_id: 72f988bf-86f1-41af-91ab-2d7cd011db47
  type: user_assigned
  user_assigned_identities:
  - client_id: db93f625-6a19-4d4e-add2-d33fedc3addf
    principal_id: 5d818a76-543c-40b6-94e2-0c35fdc24791
    resource_id: /subscriptions/72c03bf3-4e69-41af-9532-dfcdc3eefef4/resourcegroups/aml-benchmarking/providers/Microsoft.ManagedIdentity/userAssignedIdentities/finetuning-umi
  - client_id: 2ea8d340-9292-4985-9db8-0769ae8d5c91
    principal_id: 849a91ea-9b48-46f0-849d-de0f6874b370
    resource_id: /subscriptions/ed2cab61-14cc-4fb3-ac23-d72609214cfd/resourcegroups/openai-finetune-corp-rg/providers/Microsoft.ManagedIdentity/userAssignedIdentities/openai-finetune-corp-mi
idle_time_before_scale_down: 120
location: eastus
max_instances: 4
min_instances: 2
name: aoai-compute
prov

## 4. Import Components From Registry

An Azure Machine Learning component is a self-contained piece of code that does one step in a machine learning pipeline. A component is analogous to a function - it has a name, inputs, outputs, and a body. Components are the building blocks of the Azure Machine Learning pipelines. It's a good engineering practice to build a machine learning pipeline where each step has well-defined inputs and outputs. In Azure Machine Learning, a component represents one reusable step in a pipeline. Components are designed to help improve the productivity of pipeline building. Specifically, components offer:

- Well-defined interface: Components require a well-defined interface (input and output). The interface allows the user to build steps and connect steps easily. The interface also hides the complex logic of a step and removes the burden of understanding how the step is implemented.

- Share and reuse: As the building blocks of a pipeline, components can be easily shared and reused across pipelines, workspaces, and subscriptions. Components built by one team can be discovered and used by another team.

- Version control: Components are versioned. The component producers can keep improving components and publish new versions. Consumers can use specific component versions in their pipelines. This gives them compatibility and reproducibility.

For a more detailed information on this subject, refer to the this [link](https://learn.microsoft.com/en-us/azure/machine-learning/concept-component?view=azureml-api-2).

To import components,  we need to get the registry. The following command obtains the public regsitry from which we will import components for our experiment.

In [17]:
azureml_preview_registry = MLClient(credential=credential, registry_name="azureml-1p-preview")
print(azureml_preview_registry)

MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x000001A085C46FA0>,
         subscription_id=d4d34678-c0d7-4d69-a257-366e3cb4a7d8,
         resource_group_name=registry-builtin-1p-preview-eastus,
         workspace_name=None)


Next, we pull specific components from the corresponding registires and use them to build a pipeline of steps. For the illustration of the evaluation workflow we will use the following components:

| Component name | Description  | Registry name |
|:---|:---|:---|
| **aoai_finetuning**  | Upload dataset to Azure OpenAI, perform finetuning and delete dataset from Azure OpenAI. | _azureml-1p-preview_ |
| **batch_benchmark_inference**  | Inference the input dataset. | _azureml-1--preview_ |
| **batch_resource_manager** | Dual purpose: (i) deploy fine-tuned model, and (ii) delete deployment. | _azureml-1p-preview_ |
| **benchmark_result_aggregator** | Aggregate results of perfromance and quality metrics components output. | _azureml-1p-preview_ |
| **compute_metrics** | Compute quality metrics such as accuracy scores. | _azureml-1p-preview_ |
| **compute_performance_metrics** | Compute performance metrics such as runtimes. | _azureml-1p-preview_ |
| **inference_postprocessor** | Process output of the inderence step to be used by the compute metrics components.| _azureml-1p-preview_ |

In [18]:
finetuning_component = azureml_preview_registry.components.get(name="aoai_finetuning")
print(f"Data Upload component version: {finetuning_component.version}\n---")

resource_manager_component = azureml_preview_registry.components.get( "batch_resource_manager")
print(f"Resource manager component version: {resource_manager_component.version}\n---")

batch_component = azureml_preview_registry.components.get("batch_benchmark_inference")
print(f"Batch inference component version: {batch_component.version}\n---")

compute_metrics_component = azureml_preview_registry.components.get("compute_metrics")
print(f"Compute quality metrics component version: {compute_metrics_component.version}\n---")

compute_perf_metrics_component = azureml_preview_registry.components.get("compute_performance_metrics")
print(f"Compute performance metrics component version: {compute_perf_metrics_component.version}\n---")

postprocessor_component = azureml_preview_registry.components.get("inference_postprocessor")
print(f"Postprocessor component version: {postprocessor_component.version}\n---")

result_aggregator_component = azureml_preview_registry.components.get("benchmark_result_aggregator")
print(f"Results aggregator component version: {result_aggregator_component.version}\n---")

Data Upload component version: 0.0.7
---
Resource manager component version: 0.0.5
---
Batch inference component version: 0.0.9
---
Compute quality metrics component version: 0.0.26
---
Compute performance metrics component version: 0.0.6
---
Postprocessor component version: 0.0.7
---
Results aggregator component version: 0.0.8
---


## 4. Data

The component supports three methods of providing training and validation data input:
1. Direct Dataset Provisioning: Users can input data assets directly via the training_file_path and validation_file_path ports. The component will then load the data and upload it to the AOAI resource.
2. Dataset URI Provisioning:  Alternatively, users can provide dataset URIs via the `training_import_path` and `validation_import_path` ports. For this method to work, data should be accesible via GET request to the uri without requiring additional permissions. If the dataset uri is public and does not contains any credentials, user can pass it against the `data_uri` key in the training_import_path/validation_import_path json. However if the uri should not be exposed, users must first upload the data uri to the user workspaces' associated keyvault and then pass key vault key against `keyvault_key_for_data_uri` key in training_import_path/validation_import_path json

Note that exactly one of either `training_file_path` or `training_import_path` must be provided. Providing validation dataset is optional.

Along with `training_file_path` user can provide `validation_file_path`. If the latter is not provided, the training dataset will be automatically split in an 80:20 ratio to create validation data.

Along with `training_import_path`, user can provide `validation_import_path`. In the import_path json exactly one of the fields `data_uri` or `keyvault_key_for_data_uri` must be present. Since data is not loaded in component in case uri is provided, training data will not be split in the absence of validation_import_path

In the next cell we define train and validation data which will be used to fine-tune an AOAI model. We also define an inference dataset to be used to test the fine-tuned model.

In [19]:
training_file_path = ml_client.data.get(name="aoai_finetune_train", version="1")
validation_file_path = ml_client.data.get(name="aoai_finetune_validation", version="1")
inference_file_path = ml_client.data.get(name="aoai_finetune_test", version="1")

Alternatively we can also define training_import_path and validation_import_path

In [20]:
training_import_path = ml_client.data.get(name="training_import_path", version="1")
validation_import_path = ml_client.data.get(name="training_import_path", version="1")


The input data has `prompt` column which will be sent to the endpoint for batch inference and `completion` column which the ground truth.

Next, we need to set the input pattern for the batch inference step. This pattern is a string that contains the following fields:

|Parameter|Description|
|-|-|
| **messages**| This is the input prompt or prompts that are fed into the language model for generating text. The prompt can be a single sentence or multiple sentences, and it provides the initial context or topic for the generated text. It is a placeholder that indicates where the prompt should be inserted in the batch input pattern.  |
| **temperature** | this parameter controls the degree of randomness and creativity in the generated text. We set this value of 0 for reproducibility of the results. Otherwise, there will be differences in scores from run to run on the same model depending on how the data is sampled. It selects the most likely tokens until the cumulative probability exceeds a certain threshold, and then samples from those tokens based on their probabilities. |
| **max_tokens** | specifies the maximum number of tokens (words or subwords) that the model can generate in response to a given prompt. |
| **top_p** | This is another parameter used in the generation of text by language models like ChatGPT. It stands for "top probability" and controls the diversity of the generated text. This parameter is useful for controlling the length of the generated text and preventing the model from generating excessively long or irrelevant responses. |
| **frequency_penalty** | This parameter is used in some language models to discourage the repetition of the same tokens in the generated text. Specifically, it penalizes the probability of selecting a token that has already been used in the generated text. A higher frequency penalty value will result in less repetition in the generated text, while a lower value will allow for more repetition. |
| **presence_penalty** | Similar to the `frequency_penalty`, this is another parameter used in some language models to encourage the generation of new and diverse tokens (words or subwords) in the generated text. Specifically, it penalizes the probability of selecting a token that has already been used in the prompt or in the generated text. A higher presence penalty value will result in more unique and diverse responses, while a lower value will allow for more repetition. |
| **stop** |  The value of this parameter is used to specify a sequence of tokens that should be used to signal the end of the generated text. When the language model generates text, it will continue generating tokens (words or subwords) until it encounters one of the tokens specified in the "stop" parameter. |


In [21]:
batch_input_pattern = '{"messages": ###<prompt>, "temperature": 0.0, "max_tokens": 20, "top_p": 1.0, "frequency_penalty": 0.0, "presence_penalty": 0.0, "stop": null}'

Next, we specify two additional parameters `label_column_name` and `find_first`. The first parameter is just the name of the ground truth field. The `find_first` parameter is used by the post-processor component, which processes the output of the model inference step.  When specified, it must be a list of strings to search for in the inference results. The first occurrence of each string will be extracted and the occurrence with minimum index will be returned. In our dataset the objective is to select the best of the proposed choices "A,B,C,D,E". Hence, we set this value accordingly.

In [22]:
label_column_name = "completion"
find_first = "A,B,C,D,E"

## 6. Build a pipeline

Next, we build a pipeline from the imported components. Since this notebook is designed to illustrate the evaluation flow, we will string these components in the following fashion. First, we finetune and deploy an AOAI model. Next, we inference it and delete the deployment . After that, we postprocess  inference output and compute metrics. You do not have to modify anything in the next cell if you want to an exisiting postprocessing and metric computation components. 

<font color='blue' size=3 face='Verdana'>To calulate custom metrics, wrap your logic in a component, import it and modify the pipeline in the following cell. In this cenario, steps 5-8 may not be needed.</font>

In [26]:
@pipeline(description="aoai_deployment")
def aoai_deployment_with_metrics(
    training_file_path,
    validation_file_path,
    training_import_path,
    validation_import_path,
    inference_file_path,
    batch_input_pattern,
    compute_name,
    model_version=1,
    label_column_name=None,
    n_samples=10,
    handle_response_failure="neglect",
    fallback_value=None,
    additional_headers=None,
    ensure_ascii=False,
    endpoint_subscription=None,
    endpoint_resource_group=None,
    endpoint_region=None,
    deployment_sku=None,
    endpoint_name=None,
    deployment_name=None,
    max_retry_time_interval=10,
    initial_worker_count=2,
    max_worker_count=5,
    instance_count=1,
    max_concurrency_per_instance=1,
    debug_mode=False,
    do_quota_validation=False,
    use_max_quota=True,
    is_finetuned_model=False,
    finetuned_subscription_id=None,
    finetuned_resource_group=None,
    finetuned_workspace=None,
    delete_managed_deployment=True,
    remove_prefixes=None,
    separator=None,
    find_first=None,
    extract_number=None,
    regex_expr=None,
    strip_characters=None,
    label_map=None,
    template=None,
    evaluation_config_params=None,
    openai_config_params=None,
    connections_name=None,
    model="gpt-35-turbo",
    task_type="chat",
    n_epochs=1,
    batch_size=8,
    learning_rate_multiplier=1,
    suffix = None,       
    n_ctx=4096,
    lora_dim=1,
    weight_decay_multiplier=0.001,
):
    # Step 1 : Finetune OAI model
    finetune_step = finetuning_component(
        training_file_path = training_file_path,
        validation_file_path = validation_file_path,
        training_import_path = training_import_path,
        validation_import_path = validation_import_path,
        endpoint_name=endpoint_name,
        endpoint_subscription = endpoint_subscription,
        endpoint_resource_group = endpoint_resource_group,
        task_type=task_type,
        model=model,
        n_epochs=n_epochs,
        batch_size=batch_size,
        learning_rate_multiplier=learning_rate_multiplier,
        suffix = suffix,
        n_ctx=n_ctx,
        lora_dim=lora_dim,
        weight_decay_multiplier=weight_decay_multiplier
    )
    
    finetune_step.compute = compute_name

    # 2. Fine-tuned model deployment
    deployment_step = resource_manager_component(
        delete_managed_deployment=delete_managed_deployment,
        endpoint_subscription_id=endpoint_subscription,
        endpoint_resource_group=endpoint_resource_group,
        endpoint_location=endpoint_region,
        deployment_sku=deployment_sku,
        do_quota_validation=do_quota_validation,
        use_max_quota=use_max_quota,
        is_finetuned_model=is_finetuned_model,
        finetuned_subscription_id=finetuned_subscription_id,
        finetuned_resource_group=finetuned_resource_group,
        finetuned_workspace=finetuned_workspace,
        model=model,
        model_version=model_version,
        model_type="oai",
        deletion_model=False,
        deployment_name=deployment_name,
        connections_name=connections_name,
        endpoint_name=endpoint_name,
        wait_finetuned_step = False,
        finetuned_model_metadata = finetune_step.outputs.aoai_finetuning_output
    )

    deployment_step.compute = compute_name

    # 3. Batch inference. Pipeline component which deploys the model before inference.
    batch_step = batch_component(
        input_dataset=inference_file_path,
        batch_input_pattern=batch_input_pattern,
        label_column_name=label_column_name,
        deployment_name=deployment_name,
        endpoint_url="dummy",
        model_type="oai",
        n_samples=n_samples,
        handle_response_failure=handle_response_failure,
        fallback_value=fallback_value,
        additional_headers=additional_headers,
        ensure_ascii=ensure_ascii,
        max_retry_time_interval=max_retry_time_interval,
        instance_count=instance_count,
        initial_worker_count=initial_worker_count,
        max_worker_count=max_worker_count,
        max_concurrency_per_instance=1,
        authentication_type = "managed_identity",
        debug_mode=debug_mode,
        endpoint_config_file=deployment_step.outputs.output_metadata,
        connections_name="dummy"
    )
    batch_step.compute = compute_name
    batch_step.outputs.predictions = Output(
        type="uri_file",
        path="azureml://datastores/${{default_datastore}}/paths/azureml/${{name}}/${{output_name}}.jsonl",
    )
    batch_step.outputs.performance_metadata = Output(
        type="uri_file",
        path="azureml://datastores/${{default_datastore}}/paths/azureml/${{name}}/${{output_name}}.jsonl",
    )
    batch_step.outputs.ground_truth = Output(
        type="uri_file",
        path="azureml://datastores/${{default_datastore}}/paths/azureml/${{name}}/${{output_name}}.jsonl",
    )

    # 4. Delete deployment
    delete_step = resource_manager_component(
        deployment_metadata=deployment_step.outputs.output_metadata,
        wait_input=batch_step.outputs.predictions,
        delete_managed_deployment=delete_managed_deployment,
    )
    delete_step.compute = compute_name

    # 5. Comute performance metrics
    compute_perf_metrics_step = compute_perf_metrics_component(
        percentiles="50,90,99",
        batch_size_column_name="batch_size",
        start_time_column_name="start_time_iso",
        end_time_column_name="end_time_iso",
        input_token_count_column_name="input_token_count",
        output_token_count_column_name="output_token_count",
        performance_data=batch_step.outputs.performance_metadata,
    )
    compute_perf_metrics_step.compute = compute_name
    compute_perf_metrics_step.outputs.performance_result = Output(
        type="uri_file",
        path="azureml://datastores/${{default_datastore}}/paths/azureml/${{name}}/${{output_name}}.json",
    )

    # 6. Process inference output
    postprocessor_step = postprocessor_component(
        prediction_column_name="prediction",
        ground_truth_column_name=label_column_name,
        remove_prefixes=remove_prefixes,
        separator=separator,
        find_first=find_first,
        extract_number=extract_number,
        regex_expr=regex_expr,
        strip_characters=strip_characters,
        label_map=label_map,
        template=template,
        prediction_dataset=batch_step.outputs.predictions,
        ground_truth_dataset=batch_step.outputs.ground_truth,
    )
    postprocessor_step.compute = compute_name
    postprocessor_step.outputs.output_dataset_result = Output(
        type="uri_file",
        path="azureml://datastores/${{default_datastore}}/paths/azureml/${{name}}/${{output_name}}.jsonl",
    )

    # 7. Compute accuracy metrics
    compute_metrics_step = compute_metrics_component(
        task="question-answering",
        prediction_column_name="prediction",
        ground_truth_column_name=label_column_name,
        evaluation_config_params=evaluation_config_params,
        openai_config_params=openai_config_params,
        prediction=postprocessor_step.outputs.output_dataset_result,
        ground_truth=postprocessor_step.outputs.output_dataset_result,
    )
    compute_metrics_step.compute = compute_name
    compute_metrics_step.outputs.evaluation_result = Output(
        type="uri_file",
        path="azureml://datastores/${{default_datastore}}/paths/azureml/${{name}}/${{output_name}}.json",
    )

    # 8. Aggregate metrics
    aggregate_step = result_aggregator_component(
        performance_metrics=compute_perf_metrics_step.outputs.performance_result,
        quality_metrics=compute_metrics_step.outputs.evaluation_result,
    )
    aggregate_step.compute = compute_name

    return {"benchmark_result": aggregate_step.outputs.benchmark_result}

## 7. Kick Off Pipeline Runs

### 7.1. Pipeline Inputs
In the previous sections, we have defined the `input_pattern`, `label_column_name` and `inference_data` parameters. We also need to define `model_name`, `model_version` and `task` parameters.

The `task` parameter governs the task your model performs. Currently, we support `question-answering` and `chat`. In our example we ask the model to answer the questions. You may need to change this for your use case.

The `model_name` parameter specifies the name under which the fine-tuned model will be registered. The `model_version` specifies the version if more than one models will be registered under the same name. If not specified, the latest version will be used. If you specify the version, make sure it is a string. For example, model version 5 should be entered as `"5"` and not `5`.

In [27]:
endpoint_name="aoai-proxy"
endpoint_subscription="72c03bf3-4e69-41af-9532-dfcdc3eefef4"
endpoint_resource_group="aml-benchmarking"
deployment_name= "deployment_1"
task = "chat"  # "question-answering"
model_name = "gpt-35-turbo-0613"
suffix = "testing"
model_name = "gpt-35-turbo-0613"
suffix = "testing"
n_epochs=1
batch_size=8
learning_rate_multiplier=1
n_ctx=4096
lora_dim=1
weight_decay_multiplier=0.001

<!--- Another important input in the evaluation pipeline is the `endpoint_region`. It accepts a string of regions and iterates over them until it finds a region that has quota to run the pipeline. For illustration purposes we use the following regions. Your use case may be different and you will need to modify this list accordingly.

# regions = "eastus,westus,eastus2,southcentralus,centralus,northcentralus,australiaeast,canadaeast,francecentral,japaneast,swedencentral,uksouth"
# regions = None
--->

### 7.2. Kick Off the Evaluation Pipeline

In [30]:
"""Uploading data with training_file_path"""
aoai_pipeline = aoai_deployment_with_metrics(
    training_file_path=training_file_path,
    validation_file_path=validation_file_path,
    inference_file_path=inference_file_path,
    batch_input_pattern=batch_input_pattern,
    model=model_name,
    suffix=suffix,
    deployment_name= deployment_name,
    compute_name=cpu_compute_target,
    label_column_name=label_column_name,
    is_finetuned_model=True,
    delete_managed_deployment=True,
    find_first=find_first,
    task_type=task,
    n_epochs=n_epochs,
    batch_size=batch_size,
    learning_rate_multiplier=learning_rate_multiplier,       
    n_ctx=n_ctx,
    lora_dim=lora_dim,
    weight_decay_multiplier=weight_decay_multiplier,
    endpoint_name=endpoint_name,
    endpoint_subscription=endpoint_subscription,
    endpoint_resource_group=endpoint_resource_group
)

aoai_pipeline.display_name = "aoai-finetuning-with-data-asset"
aoai_pipeline.settings.default_compute = cpu_compute_target
aoai_pipeline.tags = {}
pipeline_submitted_job_base = ml_client.jobs.create_or_update(
    aoai_pipeline,
    experiment_name="aoai-finetuning-with-data",
    skip_validation=True,
    compute=cpu_compute_target,
)
ml_client.jobs.stream(pipeline_submitted_job_base.name)

RunId: bright_floor_wshg70jhgg
Web View: https://ml.azure.com/runs/bright_floor_wshg70jhgg?wsid=/subscriptions/72c03bf3-4e69-41af-9532-dfcdc3eefef4/resourcegroups/aml-benchmarking/workspaces/chirag-ws

Streaming logs/azureml/executionlogs.txt

[2024-05-16 11:31:09Z] Submitting 2 runs, first five are: 616794ef:7ef478b6-47ca-4ccc-bee6-5100a2d71df5,64829cec:b6fbbecd-3e0d-4db9-9c63-04c4b6d65fa6


JobException: The output streaming for the run interrupted.
But the run is still executing on the compute target. 
Details for canceling the run can be found here: https://aka.ms/aml-docs-cancel-run

In [31]:
"""Uploading data with training_file_key_uri"""
aoai_pipeline = aoai_deployment_with_metrics(
    training_import_path=training_import_path,
    validation_import_path=validation_import_path,
    inference_file_path=inference_file_path,
    batch_input_pattern=batch_input_pattern,
    model=model_name,
    suffix=suffix,
    deployment_name= deployment_name,
    compute_name=cpu_compute_target,
    label_column_name=label_column_name,
    is_finetuned_model=True,
    delete_managed_deployment=True,
    find_first=find_first,
    task_type=task,
    n_epochs=n_epochs,
    batch_size=batch_size,
    learning_rate_multiplier=learning_rate_multiplier,       
    n_ctx=n_ctx,
    lora_dim=lora_dim,
    weight_decay_multiplier=weight_decay_multiplier,
    endpoint_name=endpoint_name,
    endpoint_subscription=endpoint_subscription,
    endpoint_resource_group=endpoint_resource_group
)

aoai_pipeline.display_name = "aoai-finetuning-with-data-uri"
aoai_pipeline.settings.default_compute = cpu_compute_target
aoai_pipeline.tags = {}
pipeline_submitted_job_base = ml_client.jobs.create_or_update(
    aoai_pipeline,
    experiment_name="aoai-finetuning-training-data",
    skip_validation=True,
    compute=cpu_compute_target,
)
ml_client.jobs.stream(pipeline_submitted_job_base.name)

RunId: nice_piano_jbfj9hf8r9
Web View: https://ml.azure.com/runs/nice_piano_jbfj9hf8r9?wsid=/subscriptions/72c03bf3-4e69-41af-9532-dfcdc3eefef4/resourcegroups/aml-benchmarking/workspaces/chirag-ws

Streaming logs/azureml/executionlogs.txt

[2024-05-16 11:32:06Z] Submitting 2 runs, first five are: 6fd10535:fb60107e-0599-4158-b3c1-ea5147ae885d,cdec53d5:5cf4c50e-b94d-4ae9-9c59-7bad9108da5f


JobException: The output streaming for the run interrupted.
But the run is still executing on the compute target. 
Details for canceling the run can be found here: https://aka.ms/aml-docs-cancel-run

In [ ]:
ml_client.jobs.stream(pipeline_submitted_job_base.name)